In [ ]:
import os
import pandas as pd
import folium
from folium import plugins
from IPython.display import HTML,display,IFrame

class Make_Traffic_Map():
    def __init__(self,data_saved_dir:str=''):
        '''
        Make_Traffic_Map uses the data files in the data_saved_file which was previously made
        using Make_Traffic_Data class.
        --------------------------------
        data_saved_dir : parent directory which the data file exists. Contains three directory :
            1. road_Traffic_dir
            2. start_Traffic_dir
            3. end_Traffic_dir
            each directory contains two files : morning.csv, evening.csv
        --------------------------------
        '''
        if data_saved_dir[-1]!='/':
            data_saved_dir+='/'
        
        if not os.path.exists(data_saved_dir):
            raise FileNotFoundError(data_saved_file+' does not exists')
        
        self.base_dir=data_saved_dir
        
    def get_top_jam(self,what:str='road',top:int=6):
        '''
        get_top_jam shows top jamming place by road, road starting place, road ending place
        ---------------------------------------------
        what : which data to show (only takes three values "road","start","end" case insensitive)
        top : how many ranked data you want to show (default is 6)
        ---------------------------------------------
        '''
        what=what.lower()
        
        if what not in ('road','start','end'):
            raise ValueError('what must be one of these values : "road","start","end"')
            
        data_dir=self.base_dir+what+'_Traffic_dir/'
        
        print(('Showing '+what+' data').center(40,'='))
        
        for file in sorted(os.listdir(data_dir),reverse=True):
            if what=='road':
                df=pd.read_csv(data_dir+file).sort_values(by='total_jam',ascending=False)[['ROAD_NAME','start_name','end_name']]
                df.columns=['Road Name','Starting Place','Arriving Place']
                df.drop_duplicates(subset='Road Name',inplace=True)
            elif what=='start':
                df=pd.read_csv(data_dir+file).sort_values(by='total_jam',ascending=False)[['start_name','start_lat','start_lon']]
                df.columns=['Place Name','Latitude','Longitude']
                df.drop_duplicates(inplace=True)
            else:
                df=pd.read_csv(data_dir+file).sort_values(by='total_jam',ascending=False)[['end_name','end_lat','end_lon']]
                df.columns=['Place Name','Latitude','Longitude']
                df.drop_duplicates(inplace=True)
            
            print(file[:-4]+' top jammed place :')
            display(HTML(df.head(top).set_index(pd.Series(list(range(1,top+1)))).to_html()))
            
    def __save_map(self,when:str='morning',html_save_dir:str=''):
        '''
        __save_map saves a map showing the traffic jam discription.
        ---------------------------------------
        when : what time to make data ('morning','evening' case insensitive)
        html_save_dir : directory to save data
        ---------------------------------------
        '''
        when=when.lower()
        
        if when not in ('morning','evening'):
            raise ValueError('when must be one of these values : "morning","evening"')     

        road_file=self.base_dir+'road_Traffic_dir/'+when+'.csv'
        start_file=self.base_dir+'start_Traffic_dir/'+when+'.csv'
        end_file=self.base_dir+'end_Traffic_dir/'+when+'.csv'
        
        road_df=pd.read_csv(road_file)[['ROAD_NAME','start_lat','start_lon','end_lat','end_lon','total_jam','avg_speeds_avgs']]
        start_df=pd.read_csv(start_file)[['total_jam','start_lat','start_lon','start_name']]
        end_df=pd.read_csv(end_file)[['total_jam','end_lat','end_lon','end_name']]
        
        if html_save_dir[-1]!='/':
            html_save_dir+='/'
        
        if not os.path.exists(html_save_dir):
            os.mkdir(html_save_dir)
        
        daejeon=folium.Map([36.339071, 127.394635],zoom_start=12,min_zoom=12,max_zoom=16
                   ,maxBounds=[[36.5, 127.2],[36.2, 127.6]])
        
        parent=folium.FeatureGroup(name='parent',control=False)
        daejeon.add_child(parent)

        road_g=plugins.FeatureGroupSubGroup(parent,'도로표시')
        daejeon.add_child(road_g)

        start_g=plugins.FeatureGroupSubGroup(parent,'시작점기준',show=False)
        daejeon.add_child(start_g)

        end_g=plugins.FeatureGroupSubGroup(parent,'도착점기준',show=False)
        daejeon.add_child(end_g)
            
        folium.LayerControl(collapsed=False).add_to(daejeon)
        
        for _,row in road_df.iterrows():
            start=row[['start_lat','start_lon']]
            end=row[['end_lat','end_lon']]
            jam_value=row['total_jam'] / 5
            name=row['ROAD_NAME']
            speed=row['avg_speeds_avgs']
            
            if jam_value>=7.5:
                color='#FF0000'
                fillColor='#FF0000'
                fillOpacity=1
                opacity=1
                weight=10
            elif jam_value>=5:
                color='#FFFF00'
                fillColor='#FFFF00'
                fillOpacity=0.8
                opacity=0.8
                weight=5.5
            else:
                color='#FFFFFF'
                fillColor='#FFFFFF'
                fillOpacity=0
                opacity=0
                weight=0    
                
            folium.PolyLine(
            locations=[start,end],
            color=color,
            fillColor=fillColor,
            fillOpacity=fillOpacity,
            opacity=opacity,
            weight=weight,
            popup=folium.Popup(
                ' <p style=font-weight:bold;>도로명:'+name+'<br/>평균속도:'+str(speed)+'</p>',
                max_width=500
            ),
            fill=True
        ).add_to(road_g)
        
        for _,row in start_df.iterrows():
            vals=row.loc[['start_lat','start_lon','total_jam','start_name']].values
            vals[2]/=5
            
            folium.Circle(
                location=vals[:2],
                radius=vals[2]**2,
                fill=True,
                color='#ff0000',
                fillColor='#ff0000',
                opacity=0,
                fillOpacity=0.1,
                popup=folium.Popup(
                        '<p style=font-weight:bold;>'+vals[3]+'</p>',
                        max_width=500
                    )
            ).add_to(start_g)
        
        for _,row in end_df.iterrows():
            vals=row.loc[['end_lat','end_lon','total_jam','end_name']].values
            vals[2]/=5

            folium.Circle(
                location=vals[:2],
                radius=vals[2]**2,
                fill=True,
                color='#0000ff',
                fillColor='#0000ff',
                opacity=0,
                fillOpacity=0.1,
                popup=folium.Popup(
                        '<p style=font-weight:bold;>'+vals[3]+'</p>',
                        max_width=500
                    )
            ).add_to(end_g)        
        
        daejeon.save(html_save_dir+when+'.html')
        
        print('saved '+when+' data to map at '+html_save_dir+when+'.html')
        
    def save_map(self,html_save_dir:str=''):
        '''
        save_map saves the traffic jam map.
        It takes some time to make data and save them. Try to use it once
        ------------------------------------------------
        html_save_dir : the directory to save the data map
        ------------------------------------------------
        '''
        for when in ('morning','evening'):
            self.__save_map(when,html_save_dir)

    def show_map(self,when:str='morning',html_save_dir:str=''):
        '''
        show_map shows the traffic jam map which is located at the html_save_dir
        ------------------------------------------------
        when : what timeline to show data ('morning','evening' case insensitive)
        html_save_dir : the directory to save the data map
        ------------------------------------------------
        '''
        when=when.lower()

        if when not in ('morning','evening'):
            raise ValueError('when must be one of these values : "morning","evening"')

        if html_save_dir[-1]!='/':
            html_save_dir+='/'

        print((when+' traffic jam map').center(40,'-'))
        display(IFrame(html_save_dir+when+'.html',width=800,height=650))